In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

In [7]:
years = np.arange(2018, 2024)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
path_base = Path('data')
directories = [path_base / str(year) / month for year in years for month in months]

dfs = []
for dir in directories:
    if dir.exists():
        for file in dir.iterdir():
            if file.is_file() and file.name.startswith('y'):
                df = wrangle(file)
                dfs.append(df)




In [45]:
for place in each_row['Site Name'].unique():
    one_spot = each_row[each_row['Site Name']==place]
    one_spot = one_spot.dropna()

    one_spot = one_spot.to_json(orient = 'records')

    with open("city_outputs/" +place +'.json', "w") as outfile:
        json.dump(one_spot, outfile)

In [46]:
each_row = each_row.dropna()
each_row['Site Name'].unique()

array(['10Th Ave Marine Term', 'Alpine', 'Boston Ave', 'Carmel Mt Rch',
       'Chicano Park', 'Chula Vista', 'Downtown Ses', 'El Cajon Les',
       'Escondido', 'Kearny Mesa', 'Oceanview Blvd', 'Otay Mesa Dvn',
       'Pendleton', 'San Ysidro'], dtype=object)

In [24]:
bc_filt = black_carbon[black_carbon['Units'] == 'ug/m3 (LC)']
bc_filt = bc_filt[bc_filt['County'] == 'San Diego']
bc_filt.groupby('Site')['MeasuredValue'].mean()




Site
10th Ave Marine Terminal       0.693737
Caltrans at Boston Ave         0.666784
Chicano Park                   0.584152
Downtown Sherman Elementary    0.438756
Oceanview Blvd                 0.542227
Otay Mesa - Donovan            0.776540
San Ysidro                     1.182944
Name: MeasuredValue, dtype: float64

In [26]:
biggie = pd.concat(dfs)

biggie['Avg'] = biggie['Avg'].replace({'':np.nan, 'M':np.nan, '<':np.nan, 'P':np.nan})
biggie['Avg'] = biggie['Avg'].astype(float)
biggie['Site Name'] = biggie['Site Name'].str.title()
grouped  = biggie.groupby(['Site Name', 'Parameter'])['Avg'].mean()
each_row = grouped.reset_index()
each_row['Parameter'].unique()
each_row_json = each_row.to_json(orient='records')

# Write JSON to a file
with open("everything.json", "w") as outfile:
    json.dump(each_row_json, outfile)

array(['BCarbon ug/m3', '01 OZONE PPM', '02 NOX PPM', '03 NO2 PPB',
       '03 NO2 PPM', '04 NO PPB', '04 NO PPM', '10 INTMP °C',
       '11 PM2.5 µg/M3', '12 VWDR °', '13 SIGTHETA °', '14 VWSP MPH',
       '15 SWSP MPH', '16 AMBTEMP °F', '17 RELHUM %', 'PM10 STD',
       '06 CO PPM', 'BCarbon ng/m3', '18 PRES mmHg', '19 SOLAR W/M2',
       '20 UVRAD W/M2', '25 NOY', '26 NOY-NO', '27 NO Trace',
       '28 SO2 Tr PPB', '29 CO Tr PPB', 'RAINFALL', '07 H2S PPB'],
      dtype=object)

In [35]:
each_row[each_row['Parameter'] == '02 NOX PPM']

,Site Name,Parameter,Avg
2,Alpine,02 NOX PPM,0.004621
18,Carmel Mt Rch,02 NOX PPM,0.023094
30,Chula Vista,02 NOX PPM,0.010342
50,Downtown,02 NOX PPM,NaN
61,Downtown Ses,02 NOX PPM,0.013090
77,El Cajon Les,02 NOX PPM,0.011503
101,Escondido,02 NOX PPM,NaN
115,Kearny Mesa,02 NOX PPM,0.008627
133,Otay Mesa Dvn,02 NOX PPM,0.009636
148,Pendleton,02 NOX PPM,0.006795


In [6]:
import pandas as pd

# Read the file with custom parsing
file_path = '8_yesterday_20240917.csv'
def wrangle(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as file:
            lines = file.readlines()
    
    # Extract date and irrelevant lines
    date_info = lines.pop(0).strip()
    lines.pop(0)
    
    # Initialize data structure
    data = []

    # Parse the remaining lines
    for line in lines:
        split_line = line.strip().split(',')
        if len(split_line) >= 29:  # Ensure there are at least 29 fields
            data.append(split_line[:29])  # Adjust if different number of fields per row

    # Create DataFrame from the parsed data
    columns = [
        'Parameter', 'Site Name', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
        '17', '18', '19', '20', '21', '22', '23', 'Avg', 'Max', 'Hr. of Max'
    ]
    df = pd.DataFrame(data, columns=columns)
    df['Date'] =  df.iloc[0]['Parameter']
    df = df.iloc[3:]
    df['Parameter'] = df['Parameter'].replace('', np.nan)
    df['Avg'] = df['Avg'].replace({'':np.nan,'<':np.nan, 'M':np.nan, 'P':np.nan}).astype(float)

    # Forward fill the 'Parameter' column
    df['Parameter'] = df['Parameter'].ffill()
    return df
    


In [153]:
geo = pd.read_csv("2015_San_Diego_County_Demographics_-_County_Population_by_Age_and_Gender.csv")
geo[['Geography', 'Percent 65+']]


,Geography,Percent 65+
0,Central Region,9.6
1,Central San Diego,10.3
2,Mid-City,8.1
3,Southeastern San Diego,10.2
4,East Region,13.3
5,Alpine,18.5
6,El Cajon,12.3
7,Harbison Crest,18.6
8,Jamul,13.1
9,La Mesa,15.0


In [ ]:
df.merge(geo, left_on='')